# Тестирование дообученной модели 

Этот ноутбук позволяет:
- загрузить  **дообученную** модель ,
- прогнать её на STL из  теста,
- визуализировать входные рендеры,
- получить предсказанный **CAD-код/генератор** как текст.


In [ ]:
# --- Параметры ---
from pathlib import Path

MODEL_ID = "./work_dirs/cadevolve_ft/final_model"


# Один STL
STL_PATH = Path("../tests/client_01....stl")   

# Сколько вариантов сэмплить (если temperature>0)
N_SAMPLES = 1

OUT_DIR = Path("./test_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("OUT_DIR:", OUT_DIR.resolve())

In [ ]:
# --- Импорты ---
import os
import numpy as np
import torch
import trimesh

from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from qwen_vl_utils import process_vision_info
from visualization_iso import Plotter

from tqdm import tqdm
from IPython.display import display
import json

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# --- Нормализация STL: center -> 0, max_extent -> 200 (=> [-100,100]) ---
def normalize_stl_to_bbox(stl_in: Path, stl_out: Path, target_max_extent: float = 200.0):
    mesh = trimesh.load_mesh(stl_in, force="mesh")

    # иногда load_mesh возвращает Scene
    if isinstance(mesh, trimesh.Scene):
        mesh = trimesh.util.concatenate([g for g in mesh.geometry.values()])

    if mesh.vertices is None or len(mesh.vertices) == 0:
        raise ValueError(f"Empty mesh: {stl_in}")

    bounds = mesh.bounds  # (min, max)
    center = (bounds[0] + bounds[1]) / 2.0
    extents = (bounds[1] - bounds[0])
    max_extent = float(np.max(extents))

    if max_extent <= 0:
        raise ValueError(f"Degenerate mesh with max_extent={max_extent}: {stl_in}")

    scale = target_max_extent / max_extent

    # translate to origin
    mesh.apply_translation(-center)
    # scale
    mesh.apply_scale(scale)

    stl_out.parent.mkdir(parents=True, exist_ok=True)
    mesh.export(stl_out)
    return dict(center=center.tolist(), extents=extents.tolist(), max_extent=max_extent, scale=scale)

norm_stl = OUT_DIR / "normalized.stl"
stats = normalize_stl_to_bbox(STL_PATH, norm_stl, target_max_extent=200.0)
print("Normalized STL saved to:", norm_stl)
print("Norm stats:", stats)

In [ ]:
# --- Загрузка processor + model ---
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    attn_implementation="sdpa",
    trust_remote_code=True,
)

try:
    model.tie_weights()
except Exception:
    pass

model.to(device)
model.eval()
print("Loaded:", MODEL_ID)

plotter = Plotter()

def stl_to_image(stl_path: Path):
    return plotter.get_img(stl_path, None, apply_augs=False)

def build_messages(pil_img):
    return [[
        {"role": "user", "content": [{"type": "image", "image": pil_img}]},
    ]]

def generate_code_for_image(pil_img, max_new_tokens=700, temperature=0.0):
    messages = build_messages(pil_img)
    text = processor.apply_chat_template(messages[0], tokenize=False, add_generation_prompt=True)
    imgs, vids = process_vision_info(messages)

    inputs = processor(text=[text], images=imgs, videos=vids, padding=True, return_tensors="pt")
    inputs = {k: (v.to(device) if hasattr(v, "to") else v) for k, v in inputs.items()}

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature if temperature > 0 else None,
    )
    gen_kwargs = {k: v for k, v in gen_kwargs.items() if v is not None}

    with torch.no_grad():
        out = model.generate(**inputs, **gen_kwargs)

    gen = out[0][inputs["input_ids"].shape[1]:]
    pred_text = processor.tokenizer.decode(gen, skip_special_tokens=True)
    return pred_text.strip()

In [ ]:
# --- Прогон 1 STL ---
img = stl_to_image(norm_stl)
display(img)

results = []
for k in tqdm(range(N_SAMPLES)):
    pred = generate_code_for_image(img, max_new_tokens=700, temperature=0.0)
    results.append(pred)

print("Done. Samples:", len(results))
print("\nPRED (first 1200 chars):\n")
print(results[0][:1200])

In [ ]:
# --- Сохранение ---
out_json = OUT_DIR / "predictions.json"
payload = {
    "stl_in": str(STL_PATH),
    "stl_normalized": str(norm_stl),
    "normalization": stats,
    "preds": results,
}
out_json.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")

for i, pred in enumerate(results):
    (OUT_DIR / f"pred_{i:03d}.py").write_text(pred + "\n", encoding="utf-8")

print("Saved:", out_json.resolve())
print("PY files in:", OUT_DIR.resolve())